In [238]:
import pandas as pd
df = pd.read_csv("spam.csv", encoding='ISO-8859-1')

In [239]:
df.drop(columns=['Unnamed: 2',"Unnamed: 3", "Unnamed: 4"],inplace=True)

In [240]:
df.rename(columns={"v1":"label","v2":"message"}, inplace=True)

In [241]:
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [242]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
pst = PorterStemmer()
wnl = WordNetLemmatizer()

In [243]:
corpus = []
for i in range(0,len(df)):
    review = re.sub('[^a-zA-Z]'," ",df["message"][i])
    review = review.lower()
    review = review.split()
    review = [wnl.lemmatize(word) for word in review if word not in stopwords.words("english")]
    review = " ".join(review)
    corpus.append(review)

In [244]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press copy friend callertune',
 'winner valued network customer selected receivea prize reward claim call claim code kl valid hour',
 'mobile month u r entitled update latest colour mobile camera free call mobile update co free',
 'gonna home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash pound txt csh send cost p day day tsandcs apply reply hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw'

In [245]:
from sklearn.feature_extraction.text import CountVectorizer #BOW 
cv = CountVectorizer(max_features=2500 , ngram_range=(1,2))

In [246]:
#independent features 
y = pd.get_dummies(df["label"])
y = y.iloc[:,0].values

In [247]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train , y_test = train_test_split(corpus,y,test_size=0.20)

In [248]:
X_train = cv.fit_transform(X_train).toarray()
X_test = cv.transform(X_test).toarray()

In [249]:
from sklearn.naive_bayes import MultinomialNB
mb = MultinomialNB().fit(X_train,y_train)

In [250]:
y_pred = mb.predict(X_test)
y_pred

array([ True,  True,  True, ...,  True,  True,  True])

In [251]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

[[131   8]
 [  7 969]]
0.9865470852017937
              precision    recall  f1-score   support

       False       0.95      0.94      0.95       139
        True       0.99      0.99      0.99       976

    accuracy                           0.99      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [252]:
#trianing using the tfidf 
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features=2500, ngram_range=(1,2))

In [253]:
X_train, X_test, y_train, y_test = train_test_split(corpus,y,test_size=0.20)
X_train = tf.fit_transform(X_train).toarray()
X_test = tf.transform(X_test).toarray()

In [254]:
mnb = MultinomialNB().fit(X_train, y_train)
y_pred = mnb.predict(X_test)

In [255]:
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

[[131   1]
 [ 25 958]]
0.9766816143497757
              precision    recall  f1-score   support

       False       0.84      0.99      0.91       132
        True       1.00      0.97      0.99       983

    accuracy                           0.98      1115
   macro avg       0.92      0.98      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [256]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press copy friend callertune',
 'winner valued network customer selected receivea prize reward claim call claim code kl valid hour',
 'mobile month u r entitled update latest colour mobile camera free call mobile update co free',
 'gonna home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash pound txt csh send cost p day day tsandcs apply reply hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw'

In [257]:
from gensim.utils import simple_preprocess
from nltk.tokenize import sent_tokenize
words = []
for sent in corpus:
    sent_token = sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))
print(words)

[['go', 'jurong', 'point', 'crazy', 'available', 'bugis', 'great', 'world', 'la', 'buffet', 'cine', 'got', 'amore', 'wat'], ['ok', 'lar', 'joking', 'wif', 'oni'], ['free', 'entry', 'wkly', 'comp', 'win', 'fa', 'cup', 'final', 'tkts', 'st', 'may', 'text', 'fa', 'receive', 'entry', 'question', 'std', 'txt', 'rate', 'apply'], ['dun', 'say', 'early', 'hor', 'already', 'say'], ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'], ['freemsg', 'hey', 'darling', 'week', 'word', 'back', 'like', 'fun', 'still', 'tb', 'ok', 'xxx', 'std', 'chgs', 'send', 'rcv'], ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'], ['per', 'request', 'melle', 'melle', 'oru', 'minnaminunginte', 'nurungu', 'vettam', 'set', 'callertune', 'caller', 'press', 'copy', 'friend', 'callertune'], ['winner', 'valued', 'network', 'customer', 'selected', 'receivea', 'prize', 'reward', 'claim', 'call', 'claim', 'code', 'kl', 'valid', 'hour'], ['mobile', 'month', 'entitled', 'update', 'latest', 'colour', 'm

In [258]:
import gensim

In [259]:
model = gensim.models.Word2Vec(words)

In [260]:
model.epochs

5

In [261]:
#to get all the vocabulary
model.wv.index_to_key

['call',
 'get',
 'ur',
 'gt',
 'lt',
 'go',
 'ok',
 'day',
 'free',
 'know',
 'come',
 'like',
 'time',
 'good',
 'got',
 'text',
 'love',
 'want',
 'send',
 'txt',
 'need',
 'one',
 'today',
 'going',
 'home',
 'stop',
 'lor',
 'sorry',
 'see',
 'still',
 'mobile',
 'take',
 'back',
 'da',
 'reply',
 'think',
 'tell',
 'dont',
 'week',
 'phone',
 'hi',
 'new',
 'later',
 'pls',
 'please',
 'co',
 'msg',
 'min',
 'make',
 'dear',
 'night',
 'message',
 'say',
 'well',
 'thing',
 'much',
 'great',
 'claim',
 'oh',
 'hope',
 'hey',
 'number',
 'friend',
 'happy',
 'wat',
 'work',
 'give',
 'way',
 'yes',
 'www',
 'prize',
 'let',
 'right',
 'tomorrow',
 'already',
 'tone',
 'ask',
 'win',
 'said',
 'cash',
 'yeah',
 'really',
 'amp',
 'life',
 'meet',
 'im',
 'babe',
 'find',
 'morning',
 'service',
 'thanks',
 'miss',
 'year',
 'last',
 'uk',
 'com',
 'would',
 'nokia',
 'anything',
 'lol',
 'also',
 'care',
 'every',
 'feel',
 'keep',
 'pick',
 'sure',
 'sent',
 'urgent',
 'contact',


In [262]:
model.corpus_count

5564

In [263]:
model.wv.similar_by_word("moon")

[('brings', 0.9928904175758362),
 ('wife', 0.9927757382392883),
 ('phone', 0.9927682280540466),
 ('hot', 0.9927678108215332),
 ('working', 0.9927505850791931),
 ('think', 0.9927073121070862),
 ('point', 0.9926936030387878),
 ('today', 0.9926905632019043),
 ('god', 0.9926833510398865),
 ('run', 0.9926823973655701)]

In [264]:
model.wv["moon"]

array([-0.04120271,  0.05576098, -0.00397795,  0.00588805,  0.00961374,
       -0.10436764,  0.02610333,  0.10548773, -0.04995449, -0.03725287,
       -0.02795835, -0.08456869, -0.00209514,  0.04829844,  0.01140759,
       -0.04215319,  0.00551735, -0.05941353, -0.02441807, -0.12139177,
        0.02689896,  0.04859895,  0.01677966, -0.02474849, -0.03019551,
       -0.00243323, -0.04577184, -0.02640899, -0.03606123,  0.02086802,
        0.0566453 ,  0.02187854,  0.02868842, -0.03829098, -0.01489063,
        0.06085269,  0.00298869, -0.06588411, -0.05676766, -0.09633145,
       -0.01140197, -0.03961122, -0.02816547,  0.00929536,  0.05709457,
       -0.03425144, -0.01462498, -0.02798279,  0.04628528,  0.05520323,
        0.0220924 , -0.04457312, -0.00616809, -0.00533233, -0.03819039,
        0.02612642,  0.03247399, -0.01370939, -0.0781986 ,  0.01396281,
        0.03114016,  0.02700522, -0.04349714, -0.01906587, -0.08864327,
        0.05660434,  0.0275476 ,  0.02637227, -0.07659528,  0.08

In [265]:
model.wv["moon"].shape

(100,)

In [266]:
words[0] #each word in the sentence will have a (100,) dimension vectors , we use the avgword2vec to represent the whole sentence in a single
# single vector with dimension (100,) instead of using a vector for each word we use vector for each sentence this is done by taking the avg 
# of all the vectors from the word2vec  --> this is the use case of the avgword2vec

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

In [267]:
import numpy as np
from tqdm import tqdm

In [268]:
# import numpy as np
# from tqdm import tqdm

def avgword2vec(doc):
    # Filter words that are in the model's vocabulary
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)  # Compute mean along the correct axis
    else:
        return np.zeros(model.vector_size)  # Return a zero vector if no words are found

# Assuming `words` is a list of tokenized documents
X = []
for i in tqdm(range(len(words))):
    X.append(avgword2vec(words[i]))

# Convert X to a numpy array for further processing
X = np.array(X)

100%|██████████| 5564/5564 [00:00<00:00, 11924.23it/s]


In [269]:
len(X)

5564

In [270]:
X[19].shape

(100,)

In [271]:
X.shape

(5564, 100)

In [272]:
df.shape

(5572, 2)

In [273]:
#independent features 
y = pd.get_dummies(df["label"])
y = y.iloc[:,0].values
y.shape #shape of X and y are not matching this is a big concern.

(5572,)

In [274]:
[[i,j,k] for i,j,k in zip(list(map(len,corpus)),corpus,df['message']) if i < 1] #these are 8 removed from original dimesions due to the regex 
#code in which we replaced all other characters except alphabets with blank spaces.

[[0, '', 'What you doing?how are you?'],
 [0, '', 'Where @'],
 [0, '', '645'],
 [0, '', 'Can a not?'],
 [0, '', ':) '],
 [0, '', 'What you doing?how are you?'],
 [0, '', ':( but your not here....'],
 [0, '', ':-) :-)']]

In [275]:
y = df[list(map(lambda x: len(x)>0, corpus))]
y = pd.get_dummies(y["label"])
y = y.iloc[:,0].values

In [276]:
y.shape

(5564,)

In [277]:
X[1]

array([-1.11810133e-01,  2.06166953e-01, -2.77630519e-03,  5.58581986e-02,
        6.01650625e-02, -3.38652611e-01,  9.85114202e-02,  3.33089530e-01,
       -1.55888110e-01, -9.84226242e-02, -1.04931727e-01, -2.81810701e-01,
       -2.59302203e-02,  1.33370250e-01,  2.91350968e-02, -1.24028064e-01,
        3.67504880e-02, -2.35715389e-01, -7.90474713e-02, -4.23797727e-01,
        8.56660455e-02,  1.38459504e-01,  3.30683477e-02, -9.44112614e-02,
       -1.15389720e-01, -1.05300569e-03, -1.43218786e-01, -1.11677773e-01,
       -1.54839307e-01,  6.05973154e-02,  2.29086354e-01,  4.54706661e-02,
        6.85289577e-02, -1.04296178e-01, -8.68226737e-02,  1.95952773e-01,
        3.05011552e-02, -2.18544692e-01, -1.70021310e-01, -3.59335542e-01,
       -5.65701490e-03, -1.39138937e-01, -5.52010313e-02,  5.12130037e-02,
        1.90940082e-01, -1.06909171e-01, -5.52397817e-02, -7.85464346e-02,
        1.82549953e-01,  1.74628913e-01,  7.34156370e-02, -1.69683784e-01,
       -7.50223920e-03, -

In [278]:
#now we should make each sentence of 100 dimensions as a single row in the data frame so X[0] ->(100,) now it become (1,100) that is 1 row with 
#100 columns that is not but 100 features.
df = pd.DataFrame()
for i in range(0,len(X)):
    d = pd.DataFrame(X[i].reshape(1,-1))
    df = pd.concat([df, d],ignore_index=True )

In [279]:
df.shape

(5564, 100)

In [280]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.134857,0.252080,0.000009,0.056656,0.074353,-0.402554,0.122800,0.403467,-0.184304,-0.122354,...,0.309624,0.097658,-0.047578,0.075712,0.422250,0.217580,0.133682,-0.304195,0.148759,-0.000057
1,-0.111810,0.206167,-0.002776,0.055858,0.060165,-0.338653,0.098511,0.333090,-0.155888,-0.098423,...,0.256063,0.083678,-0.044227,0.062608,0.356629,0.187322,0.122308,-0.254346,0.131138,-0.000376
2,-0.141835,0.271780,-0.005957,0.066660,0.078749,-0.412919,0.132873,0.413527,-0.191758,-0.116608,...,0.315011,0.101321,-0.047593,0.082528,0.442576,0.234680,0.134063,-0.310703,0.166475,-0.001813
3,-0.192827,0.349889,-0.001943,0.086927,0.097856,-0.569438,0.172480,0.572241,-0.268791,-0.162364,...,0.439302,0.138575,-0.067832,0.107014,0.606886,0.307557,0.199613,-0.430000,0.217108,-0.011611
4,-0.156854,0.290184,0.002878,0.064401,0.082645,-0.463119,0.140170,0.463329,-0.212202,-0.136794,...,0.352925,0.116157,-0.053940,0.085155,0.480673,0.247904,0.156742,-0.344988,0.171743,-0.003143


In [ ]:
#independent features 
X = df
y

(5564,)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train , y_test = train_test_split(X,y,test_size=0.20)

In [285]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [289]:
y_pred = rfc.predict(X_test)
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score
print(accuracy_score(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.9658580413297394
[[119   7]
 [ 31 956]]
              precision    recall  f1-score   support

       False       0.79      0.94      0.86       126
        True       0.99      0.97      0.98       987

    accuracy                           0.97      1113
   macro avg       0.89      0.96      0.92      1113
weighted avg       0.97      0.97      0.97      1113

